In [ ]:
import os
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from transformers import AutoModelForImageClassification, AutoFeatureExtractor
from PIL import Image
import nltk
from nltk.translate.bleu_score import sentence_bleu

In [ ]:
import json
!pip install kaggle

In [13]:
with open("C:/Users/BT_Lab/Downloads/W25/ASLDataset/WLASLDataset/nslt_2000.json", "r") as f:
    data = json.load(f)

In [14]:
print(json.dumps(data, indent = 2))

{
  "65097": {
    "subset": "train",
    "action": [
      481,
      1,
      74
    ]
  },
  "35544": {
    "subset": "train",
    "action": [
      1164,
      1,
      68
    ]
  },
  "32962": {
    "subset": "val",
    "action": [
      415,
      1,
      43
    ]
  },
  "30173": {
    "subset": "test",
    "action": [
      1144,
      1,
      34
    ]
  },
  "35547": {
    "subset": "val",
    "action": [
      1164,
      1,
      55
    ]
  },
  "35548": {
    "subset": "train",
    "action": [
      1164,
      1,
      24
    ]
  },
  "24029": {
    "subset": "train",
    "action": [
      208,
      1,
      28
    ]
  },
  "51829": {
    "subset": "train",
    "action": [
      1929,
      1,
      54
    ]
  },
  "24028": {
    "subset": "test",
    "action": [
      208,
      1,
      55
    ]
  },
  "48027": {
    "subset": "train",
    "action": [
      969,
      1,
      55
    ]
  },
  "38842": {
    "subset": "train",
    "action": [
      1172,
      1,
      

In [17]:
import os
import shutil

# Path to the video files
video_path = "C:/Users/BT_Lab/Downloads/W25/ASLDataset/WLASLDataset/videos"  # Adjust the path accordingly
output_dir = "C:/Users/BT_Lab/Downloads/W25/ASLDataset/WLASLDataset/sorted_videos"  # Directory to organize files

# Create directories for train, val, and test
for subset in ["train", "val", "test"]:
    os.makedirs(os.path.join(output_dir, subset), exist_ok=True)

# Iterate through JSON data and move files into the corresponding subset folder
for video_id, video_info in data.items():
    subset = video_info["subset"]  # Get the subset ('train', 'val', 'test')
    video_file = f"{video_id}.mp4"  # Assuming the video file is named after the video ID
    
    # If the video file exists in the video folder, move it
    if os.path.exists(os.path.join(video_path, video_file)):
        shutil.move(os.path.join(video_path, video_file), os.path.join(output_dir, subset, video_file))

print("Dataset organized successfully!")

Dataset organized successfully!


In [18]:
with open("C:/Users/BT_Lab/Downloads/W25/ASLDataset/WLASLDataset/wlasl_class_list.txt", "r") as f:
    class_list = f.readlines()

# Clean and print first 10 words
class_list = [word.strip() for word in class_list]
print(class_list[:10])  # Show first 10 ASL glosses

['0\tbook', '1\tdrink', '2\tcomputer', '3\tbefore', '4\tchair', '5\tgo', '6\tclothes', '7\twho', '8\tcandy', '9\tcousin']


In [6]:
#read class list with pandas 
import pandas as pd 
classlist = pd.read_csv("C:/Users/BT_Lab/Downloads/W25/ASLDataset/WLASLDataset/wlasl_class_list.txt", sep= "\t")
classlist.columns = ["class",'word']

In [9]:
#read json file with pandas 

df= pd.read_json("C:/Users/BT_Lab/Downloads/W25/ASLDataset/WLASLDataset/nslt_2000.json", orient= 'index' )
print(df.index)
df[['class','participant','frames']] = pd.DataFrame(df.action.tolist(), index= df.index)
index= df.index.copy()
df= pd.merge(df, classlist, how= 'left',left_on= 'class', right_on= 'class')
df.index = index

Index([65097, 35544, 32962, 30173, 35547, 35548, 24029, 51829, 24028, 48027,
       ...
       35765, 30923, 38849, 50936, 38341, 15905, 38343, 38342, 38345, 38344],
      dtype='int64', length=21095)


In [19]:
import os
import cv2  # Assuming OpenCV for video reading
import numpy as np

# Define directories for train, val, and test sets
train_dir = "C:/Users/BT_Lab/Downloads/W25/ASLDataset/WLASLDataset/sorted_videos/train"
val_dir = "C:/Users/BT_Lab/Downloads/W25/ASLDataset/WLASLDataset/sorted_videos/val"
test_dir = "C:/Users/BT_Lab/Downloads/W25/ASLDataset/WLASLDataset/sorted_videos/test"

# Function to load videos and labels
def load_data(directory, class_list, data_type="train"):
    video_paths = []
    labels = []

    for class_name in os.listdir(directory):
        class_folder = os.path.join(directory, class_name)
        
        if os.path.isdir(class_folder):  # Ensure it's a directory (class folder)
            for video_file in os.listdir(class_folder):
                if video_file.endswith(".mp4"):  # Assuming videos are in mp4 format
                    video_path = os.path.join(class_folder, video_file)
                    video_id = int(video_file.split(".")[0])  # Extract the ID from filename
                    
                    # Get the corresponding action label (class ID)
                    action_id = data[video_id]["action"][0]  # First element of the action list
                    label = class_list.get(action_id, "Unknown")  # Get the corresponding class name
                    
                    video_paths.append(video_path)  # Store the video path
                    labels.append(label)  # Store the label for this video

    return video_paths, labels

# Load training, validation, and test data
train_video_paths, train_labels = load_data(train_dir, class_list, "train")
val_video_paths, val_labels = load_data(val_dir, class_list, "val")
test_video_paths, test_labels = load_data(test_dir, class_list, "test")

# Example: Print the first 3 entries of train data
print(train_video_paths[:3])
print(train_labels[:3])

[]
[]


In [24]:
print(data["65600"]) 
#the video is  number 65097 in the train folder of the sorted dataset 
#Result : {'subset': 'train', 'action': [481, 1, 74]}
#481 corresponds to the word ID in the Class list
#number  1 corresponds to the signer ID and 74 is the video frame 


{'subset': 'train', 'action': [277, 1, 72]}


In [22]:
print(class_list) 

['0\tbook', '1\tdrink', '2\tcomputer', '3\tbefore', '4\tchair', '5\tgo', '6\tclothes', '7\twho', '8\tcandy', '9\tcousin', '10\tdeaf', '11\tfine', '12\thelp', '13\tno', '14\tthin', '15\twalk', '16\tyear', '17\tyes', '18\tall', '19\tblack', '20\tcool', '21\tfinish', '22\thot', '23\tlike', '24\tmany', '25\tmother', '26\tnow', '27\torange', '28\ttable', '29\tthanksgiving', '30\twhat', '31\twoman', '32\tbed', '33\tblue', '34\tbowling', '35\tcan', '36\tdog', '37\tfamily', '38\tfish', '39\tgraduate', '40\that', '41\thearing', '42\tkiss', '43\tlanguage', '44\tlater', '45\tman', '46\tshirt', '47\tstudy', '48\ttall', '49\twhite', '50\twrong', '51\taccident', '52\tapple', '53\tbird', '54\tchange', '55\tcolor', '56\tcorn', '57\tcow', '58\tdance', '59\tdark', '60\tdoctor', '61\teat', '62\tenjoy', '63\tforget', '64\tgive', '65\tlast', '66\tmeet', '67\tpink', '68\tpizza', '69\tplay', '70\tschool', '71\tsecretary', '72\tshort', '73\ttime', '74\twant', '75\twork', '76\tafrica', '77\tbasketball', '78\tb

In [20]:
def process_video(video_path, frame_size=(224, 224)):
    cap = cv2.VideoCapture(video_path)
    frames = []
    
    while(cap.isOpened()):
        ret, frame = cap.read()
        if not ret:
            break
        # Resize the frame to a consistent size (e.g., 224x224)
        frame_resized = cv2.resize(frame, frame_size)
        frames.append(frame_resized)
    
    cap.release()
    return np.array(frames)

# Example: Process the first video in the training set
first_video_frames = process_video(train_video_paths[0])
print(f"Processed video shape: {first_video_frames.shape}")

IndexError: list index out of range